In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow

In [7]:
!ls
!pwd
import os
os.chdir("/content/drive/MyDrive/bitirme-projesi")

drive  sample_data
/content


In [8]:
!ls

bitirme_projesi.ipynb  sent-analysis.ipynb
hepsiburada.csv        sentiment_analysis_model.h5


In [9]:
dataset = pd.read_csv("hepsiburada.csv")

In [10]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [11]:
target = dataset["Rating"].values.tolist()

In [12]:
data = dataset["Review"].values.tolist()

In [13]:
cut_off = int(len(data) * 0.8)

In [14]:
x_train, x_test = data[:cut_off], data[cut_off:]

In [15]:
y_train, y_test = target[:cut_off], target[cut_off:]

In [17]:
num_words = 10000

tokenizer = Tokenizer(num_words=num_words)

tokenizer.fit_on_texts(data)

In [19]:
tokenizer.word_index

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [20]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [21]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [22]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]

In [23]:
num_tokens = np.array(num_tokens)

In [24]:
np.mean(num_tokens)

20.744703220162876

In [25]:
np.max(num_tokens)


295

In [26]:
np.argmax(num_tokens)

21941

In [27]:
x_train[21941]

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [28]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)

In [30]:
max_tokens = int(max_tokens)

max_tokens

59

In [31]:
np.sum(num_tokens<max_tokens)/len(num_tokens)

0.9597982726686571

In [32]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [33]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [34]:
x_train_pad.shape

(194797, 59)

In [35]:
x_test_pad.shape

(48700, 59)

In [36]:
np.array(x_train_tokens[800])

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [37]:
x_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10], dtype=int32)

In [38]:
idx = tokenizer.word_index

In [39]:
inverse_map = dict(zip(idx.values(), idx.keys()))

In [40]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = " ".join(words)
    return text

In [41]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [42]:
tokens_to_string(x_train_tokens[800])


'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

In [43]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=num_words, 
                    output_dim=embedding_size, 
                    input_length=max_tokens, 
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = tensorflow.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer= optimizer, 
              loss=tensorflow.keras.losses.binary_crossentropy , 
              metrics=['accuracy'])

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 59, 50)           500000    
                                                                 
 gru (GRU)                   (None, 59, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 59, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [45]:
history = model.fit(np.array(x_train_pad), 
                    np.array(y_train), 
                    epochs=3, 
                    batch_size=256)

Epoch 1/3
761/761 [==============================] - 17s 11ms/step - loss: 0.1639 - accuracy: 0.9551
Epoch 2/3
761/761 [==============================] - 8s 11ms/step - loss: 0.0913 - accuracy: 0.9705
Epoch 3/3
761/761 [==============================] - 8s 11ms/step - loss: 0.0726 - accuracy: 0.9765


In [51]:
result = model.evaluate(np.array(x_test_pad), np.array(y_test))

1522/1522 [==============================] - 9s 5ms/step - loss: 0.1281 - accuracy: 0.9571


In [52]:
result[1]

0.9571458101272583

In [53]:
y_pred = model.predict(x=x_test_pad[0:1000])

32/32 [==============================] - 1s 5ms/step


In [54]:
y_pred = y_pred.T[0]

In [55]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [56]:
cls_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0.

In [57]:
cls_true = np.array(y_test[0:1000])

In [58]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [59]:
len(incorrect)

57

In [60]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadaım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [61]:
tokens = tokenizer.texts_to_sequences(texts)

In [62]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(10, 59)

In [63]:
model.predict(tokens_pad)

1/1 [==============================] - 0s 18ms/step


array([[0.99562883],
       [0.9625551 ],
       [0.14055325],
       [0.9924859 ],
       [0.15528767],
       [0.14622815],
       [0.8698243 ],
       [0.06531171],
       [0.99429   ],
       [0.3326555 ]], dtype=float32)